# Download data and import libraries

In [1]:
%matplotlib inline 

import pandas as pd
import numpy as np

from pandas_profiling import ProfileReport
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score

import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
X_train = df_train[pv_selected_columns].values
y_train = df_train['Revenue'].values

X_test = df_test[pv_selected_columns].values
y_test = df_test['Revenue'].values

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(6523, 14) (6523,)
(2035, 14) (2035,)


In [19]:
#Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train_st = scaler.fit(X_train).transform(X_train)
X_test_st = scaler.transform(X_test)

# Linear models
Since labels are very skewed towards "False" hyper-parameter "class_weight" value is "balanced".

Model selection process:
1. for each model candidate find hyper-parameters that guarantee its best performance
2. among models traind using best parameters select one that delivers best recall for "Revenue"=True (Purchase) class

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [21]:
# creating recall scoribg function for CV that focuses on just "Purchase" class
recall_score2 = make_scorer(recall_score, pos_label=1)

In [22]:
# creating a dict of linear models to choose from
model_dict = {'Logistic Regression' : LogisticRegression(),
              "Linear SVM": LinearSVC()}

In [23]:
# creating a dict of parameters for the models
params_LR = dict(penalty=['l1', 'l2', 'elasticnet'],
                 C=[0.2,0.5,1,2,5,10],
                 fit_intercept=[True, False],
                 class_weight = ['balanced']
                 )
params_SVM = dict(penalty=['l1', 'l2'],
                  loss=['hinge', 'squared_hinge'],
                  C=[0.2,0.5,1,2,5,10],
                  fit_intercept=[True, False],
                  class_weight = ['balanced']
                  )
parameters_dict = {'Logistic Regression': params_LR,
                  "Linear SVM" : params_SVM}

In [24]:
# 5-fold gridsearch through parameters maximazing Recall for Purchases
model_name, model_parameters, model_best_score = [], [], []

model_names = model_dict.keys()

for MN in model_names:
    print("="*45)
    print(MN)
    clf = model_dict[MN]
    params = parameters_dict[MN]

    gridsearch = GridSearchCV(clf,
                              params,
                              scoring=recall_score2,
                              cv=5,
                              verbose=1,
                              n_jobs=-1)
    
    best_model = gridsearch.fit(X_train, y_train)
    model_name.append(MN)
    model_parameters.append(gridsearch.best_params_)
    model_best_score.append(gridsearch.best_score_ )

Logistic Regression
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 149 out of 180 | elapsed:    1.4s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    1.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


Linear SVM
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 209 out of 240 | elapsed:    3.0s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    3.3s finished
/Users/tatiana/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [25]:
# comparing different models
model_comparison_df = pd.DataFrame([model_name, model_best_score, model_parameters]).T
model_comparison_df.columns = ['model_name', 'recall','parameters']

model_comparison_df = model_comparison_df.sort_values(by='recall', ascending=False)
clf_name = model_comparison_df['model_name'].iloc[0]
clf_params = model_comparison_df['parameters'].iloc[0]
clf_accuracy = model_comparison_df['recall'].iloc[0]

print('Best model:', clf_name)
model_comparison_df[['model_name', 'recall', 'parameters']]

Best model: Logistic Regression


,model_name,recall,parameters
0,Logistic Regression,0.678992,"{'C': 2, 'class_weight': 'balanced', 'fit_inte..."
1,Linear SVM,0.548739,"{'C': 0.5, 'class_weight': 'balanced', 'fit_in..."


In [26]:
#check parameter values
model_comparison_df['parameters'].iloc[0]

{'C': 2, 'class_weight': 'balanced', 'fit_intercept': False, 'penalty': 'l2'}

Cross validation parametrs are chosen so that for any model best performance is achieved on parameter values that are not on a boundary.

In [27]:
# getting predictions from the best model
clf = model_dict[clf_name]
clf.set_params(**clf_params)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(clf_name)
print("Model performance:")
print(classification_report(y_test, y_pred))
print("Confusion matrix")
print(cm)

Logistic Regression
Model performance:
              precision    recall  f1-score   support

       False       0.98      0.92      0.95      1855
        True       0.51      0.84      0.63       180

    accuracy                           0.91      2035
   macro avg       0.75      0.88      0.79      2035
weighted avg       0.94      0.91      0.92      2035

Confusion matrix
[[1708  147]
 [  28  152]]


# Non-Linear models


In [28]:
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [29]:
# creating a dict of models
model_dict = {'Stochastic Gradient Descent' : SGDClassifier(),
              'Random Forest': RandomForestClassifier(),
              'Neural Network': MLPClassifier()
             }

In [30]:
# creating a dict of parameters for the models
params_SGD = dict(loss=['log'],
                  penalty=['l2','l1'],
                  alpha=[1e-6, 1e-3, 1e-1, 1e0],
                  max_iter=[5, 1000, 10000],
                  tol=[None, 1e-3],
                  random_state=[3])
params_RF = dict(bootstrap = [True, False],
                  max_depth = [10, 50, 100, None],
                  max_features = ['auto', 'sqrt'],
                  min_samples_leaf = [1, 2, 4],
                  n_estimators = [100, 500, 1000],
                  random_state=[3])
params_NN = dict(hidden_layer_sizes = [(100,),(150,), (200,), (250,),(500,),(750,), (1000,)],
                 alpha=[0.00001, 0.00005, 0.0001, 0.0005],
                 activation=['relu'],
                 solver=['adam'],
                  random_state=[3])

parameters_dict = {'Stochastic Gradient Descent' : params_SGD,
                   'Random Forest': params_RF,
                   'Neural Network': params_NN
                  }

In [31]:
# 5-fold gridsearch through parameters maximazing Recall for Purchases
model_name, model_parameters, model_best_score = [], [], []

model_names = model_dict.keys()

for MN in model_names:
    print("="*45)
    print(MN)
    clf = model_dict[MN]
    params = parameters_dict[MN]

    gridsearch = GridSearchCV(clf,
                              params,
                              scoring=recall_score2,
                              cv=5,
                              verbose=1,
                              n_jobs=-1)
    
    best_model = gridsearch.fit(X_train, y_train)
    model_name.append(MN)
    model_parameters.append(gridsearch.best_params_)
    model_best_score.append(gridsearch.best_score_ )

Stochastic Gradient Descent
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 209 out of 240 | elapsed:   18.9s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   31.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


Random Forest
Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   40.6s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.7min
/Users/tatiana/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  3.4min finished


Neural Network
Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.5s
/Users/tatiana/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:   36.5s finished


In [32]:
# comparing different models
model_comparison_df = pd.DataFrame([model_name, model_best_score, model_parameters]).T
model_comparison_df.columns = ['model_name', 'recall','parameters']

model_comparison_df = model_comparison_df.sort_values(by='recall', ascending=False)
clf_name = model_comparison_df['model_name'].iloc[0]
clf_params = model_comparison_df['parameters'].iloc[0]
clf_accuracy = model_comparison_df['recall'].iloc[0]

print('Best model:', clf_name)
model_comparison_df[['model_name', 'recall', 'parameters']]

Best model: Neural Network


,model_name,recall,parameters
2,Neural Network,0.689076,"{'activation': 'relu', 'alpha': 5e-05, 'hidden..."
0,Stochastic Gradient Descent,0.568067,"{'alpha': 0.001, 'loss': 'log', 'max_iter': 10..."
1,Random Forest,0.490756,"{'bootstrap': True, 'max_depth': 50, 'max_feat..."


In [33]:
#check parameter values
model_comparison_df['parameters'].iloc[0]

{'activation': 'relu',
 'alpha': 5e-05,
 'hidden_layer_sizes': (500,),
 'random_state': 3,
 'solver': 'adam'}

Cross validation parametrs are chosen so that for any model best performance is achieved on parameter values that are not on a boundary.

In [34]:
# getting predictions from the best model
clf = model_dict[clf_name]
clf.set_params(**clf_params)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(clf_name)
print("Model performance:")
print(classification_report(y_test, y_pred))
print("Confusion matrix")
print(cm)

Neural Network
Model performance:
              precision    recall  f1-score   support

       False       0.99      0.91      0.95      1855
        True       0.50      0.93      0.65       180

    accuracy                           0.91      2035
   macro avg       0.75      0.92      0.80      2035
weighted avg       0.95      0.91      0.92      2035

Confusion matrix
[[1691  164]
 [  13  167]]


# Classification performance on data entries corresponding to Feb-March:
**Best Models:**

|         Classifier        | Accuracy |Precision| Recall | F1-score |
|---------------------------|----------|---------|--------|----------|
**Linear Classifier**
|        Logistic Regression|   0.91   |  0.51   |  0.84  |  0.63    |
**Non Linear Classiifer**
|             Neural Network|   0.91   |  0.50   |  0.93  |  0.65    |


Note: Precision, Recall and F1-score are measured for "purchase" class

